In [1]:
pip install --upgrade tensorflow==2.1.0

     |████████████████████████████████| 421.8MB 24kB/s 
     |████████████████████████████████| 450kB 38.0MB/s 
     |████████████████████████████████| 3.9MB 5.5MB/s 
     |████████████████████████████████| 51kB 7.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=c14973d8d76ffcdeb206716431e1136e59b5d5bda74813798ef1e04efde65899
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstal

In [2]:
pip install --upgrade keras==2.3.1

     |████████████████████████████████| 378kB 7.7MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


In [1]:
pip list

Package                       Version        
----------------------------- ---------------
absl-py                       0.8.1          
alabaster                     0.7.12         
albumentations                0.1.12         
altair                        4.1.0          
argon2-cffi                   20.1.0         
asgiref                       3.2.10         
astor                         0.8.1          
astropy                       4.0.1.post1    
astunparse                    1.6.3          
atari-py                      0.2.6          
atomicwrites                  1.4.0          
attrs                         20.1.0         
audioread                     2.1.8          
autograd                      1.3            
Babel                         2.8.0          
backcall                      0.2.0          
beautifulsoup4                4.6.3          
bleach                        3.1.5          
blis                          0.4.1          
bokeh                         2.1.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
!ls 'drive/My Drive'

'Colab Notebooks'   datasets   generate_imgs   saved_model


In [3]:
import numpy as np
import glob
import os
import cv2
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import keras as K
from keras.layers import Activation, BatchNormalization, Dense, Dropout, Flatten, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential
from keras.optimizers import Adam


Using TensorFlow backend.


In [4]:
def build_generator(z_dim):

    model = Sequential()

    # 8x8x128
    model.add(Dense(8 * 8 * 128, input_dim=z_dim))
    model.add(Reshape((8, 8, 128)))

    # 8x8x128 => 16x16x64
    model.add(Conv2DTranspose(64, kernel_size=3, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.1))
    # 16x16x64 => 32x32x32
    model.add(Conv2DTranspose(32, kernel_size=3, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.1))
    # 32x32x32 => 64x64x16
    model.add(Conv2DTranspose(16, kernel_size=3, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.1))
    # 64x64x16 => 128x128x16
    model.add(Conv2DTranspose(16, kernel_size=3, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.1))
    # 128x128x16 => 128x128x3
    model.add(Conv2D(3, kernel_size=3, strides=1, padding='same'))

    # tanh関数を適用して出力
    model.add(Activation('tanh'))

    return model

In [5]:
def build_discriminator(img_shape):

    model = Sequential()

    # 128x128x3 => 64x64x16
    model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=img_shape, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(0.25))

    # 64x64x16 => 32x32x32
    model.add(Conv2D(32, kernel_size=3, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(0.25))

    # 32x32x32 => 16x16x32
    model.add(Conv2D(32, kernel_size=3, strides=2, padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.1))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    return model

In [6]:
def build_gan(generator, discriminator):

    model = Sequential()

    model.add(generator)
    model.add(discriminator)

    return model


In [12]:
def train(iterations, batch_size, sample_interval):

    losses = []
    accuracies = []
    iteration_checkpoints = []

    dataset = np.load('drive/My Drive/datasets/irasutoya_re_128.npz')

    x_train = np.array(dataset['X_train'])

     # 本物の画像のラベルは全て1にする
    real = np.ones((batch_size, 1))

    # 偽物の画像のラベルは全て0にする
    fake = np.zeros((batch_size, 1))

    for iteration in range(iterations):

        #-------------
        # 識別器の学習
        #-------------

        # 本物の画像集合からランダムにバッチを生成する
        idx = np.random.randint(0, x_train.shape[0], batch_size)
        imgs = x_train[idx]

        # 偽物の画像からなるバッチを生成する
        z = np.random.normal(0, 1, (batch_size, z_dim))
        gen_imgs = generator.predict(z)

        # 識別器の学習
        d_loss_real = discriminator.train_on_batch(imgs, real)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss, accuracy = 0.5 * np.add(d_loss_real , d_loss_fake)

        #-------------
        # 生成器の学習
        #-------------
        for i in range(2):
            # ノイズベクトルを生成
            z = np.random.normal(0, 1, (batch_size, z_dim))    
            # 生成器の学習
            g_loss = gan.train_on_batch(z, real)

        print('\rNo, %d' %(iteration+1), end='')

        if (iteration + 1) % sample_interval == 0:

            # あとで可視化するために損失と精度を保存しておく
            losses.append([d_loss, g_loss])
            accuracies.append(100.0 * accuracy)
            iteration_checkpoints.append(iteration + 1)

            # 学習結果の出力
            print('[D loss: %f, acc.: %.2f%%] [G loss: %f]' %(d_loss, 100.0*accuracy, g_loss))

            sample_images(generator, iteration)

        if (iteration + 1) % (sample_interval * 2) == 0:
            generator.save("drive/My Drive/saved_model/irasutoya/original_64_2/%d_IRASUTOYA.h5" % (iteration+1))


In [11]:
def sample_images(generator, iteration, image_grid_rows=4, image_grid_columns=4):

    # ノイズベクトルを生成する
    z = np.random.normal(0, 1, (image_grid_rows * image_grid_columns, z_dim))

    # ノイズベクトルから画像を生成する
    gen_imgs = generator.predict(z)

    # 出力の画素値を[0, 1]の範囲にスケーリングする
    gen_imgs = 0.5 * gen_imgs + 0.5


    # 画像からなるグリッドを生成する
    fig, axs = plt.subplots(image_grid_rows, image_grid_columns, figsize=(4, 4), sharey=True, sharex=True)
    count = 0
    for i in range(image_grid_rows):
        for j in range(image_grid_columns):
            # 並べた画像を出力
            axs[i, j].imshow(gen_imgs[count, :, :, :])
            axs[i, j].axis('off')
            count += 1
    
    fig.savefig("drive/My Drive/generate_imgs/irasutoya/original_64_2/%d.png" % (iteration+1))
    plt.close()

In [ ]:
img_rows = 128
img_cols = 128
channels = 3

iterations = 1000000
batch_size = 32
sample_interval = 500

img_shape = (img_rows, img_cols, channels)
z_dim = 100

# 識別器
discriminator = build_discriminator(img_shape)
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# 生成器
generator = build_generator(z_dim)

discriminator.trainable = False

gan = build_gan(generator, discriminator)
gan.compile(optimizer=Adam(), loss='binary_crossentropy')

train(iterations, batch_size, sample_interval)

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


No, 1

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


No, 500[D loss: 0.351390, acc.: 82.81%] [G loss: 1.233285]
No, 1000[D loss: 0.212178, acc.: 90.62%] [G loss: 1.501391]
No, 1500[D loss: 0.180252, acc.: 96.88%] [G loss: 1.195703]
No, 2000[D loss: 0.853280, acc.: 60.94%] [G loss: 1.525445]
No, 2500[D loss: 0.071190, acc.: 98.44%] [G loss: 0.736027]
No, 3000[D loss: 0.211522, acc.: 92.19%] [G loss: 1.726943]
No, 3500[D loss: 0.098866, acc.: 96.88%] [G loss: 2.652227]
No, 4000[D loss: 0.355774, acc.: 79.69%] [G loss: 1.726734]
No, 4500[D loss: 0.457491, acc.: 81.25%] [G loss: 1.701625]
No, 5000[D loss: 0.138261, acc.: 93.75%] [G loss: 3.644284]
No, 5500[D loss: 0.645752, acc.: 67.19%] [G loss: 1.098034]
No, 6000[D loss: 0.545153, acc.: 68.75%] [G loss: 0.871337]
No, 6500[D loss: 0.548352, acc.: 70.31%] [G loss: 2.035631]
No, 7000[D loss: 0.300241, acc.: 85.94%] [G loss: 1.873002]
No, 7500[D loss: 0.478229, acc.: 78.12%] [G loss: 2.437812]
No, 8000[D loss: 0.265179, acc.: 90.62%] [G loss: 3.231730]
No, 8500[D loss: 0.096801, acc.: 96.88%] 